In [1]:
import numpy as np

import astropy.units as u
from astropy.time import Time
from astropy.coordinates import solar_system_ephemeris
solar_system_ephemeris.set("jpl")

from poliastro.bodies import Sun, Earth, Jupiter, Venus
from poliastro.ephem import Ephem
from poliastro.frames import Planes
from poliastro.twobody import Orbit
from poliastro.maneuver import Maneuver
from poliastro.plotting import StaticOrbitPlotter
from poliastro.util import norm, time_range
from poliastro import iod
from poliastro.plotting import OrbitPlotter3D
from poliastro.threebody.flybys import compute_flyby

import plotly.io as pio
pio.renderers.default = "notebook_connected"

In [2]:
## Main dates (4 набор дат)
date_launch = Time("2026-06-07", scale="utc").tdb
date_flyby_Venus = Time("2026-11-21", scale="utc").tdb
date_flyby_Earth = Time("2028-09-09", scale="utc").tdb
date_arrival = Time("2030-10-19", scale="utc").tdb

In [3]:
earth_1 = Ephem.from_body(Earth, time_range(date_launch, end=date_launch))
ven = Ephem.from_body(Venus, time_range(date_launch, end=date_flyby_Venus))
earth_2 = Ephem.from_body(Earth, time_range(date_launch, end=date_flyby_Earth))
jup = Ephem.from_body(Jupiter, time_range(date_launch, end=date_arrival))

In [4]:
ss_earth_1 = Orbit.from_ephem(Sun, earth_1, date_launch)
ss_ven = Orbit.from_ephem(Sun, ven, date_flyby_Venus)
ss_earth_2 = Orbit.from_ephem(Sun, earth_2, date_flyby_Earth)
ss_jup = Orbit.from_ephem(Sun, jup, date_arrival)

In [5]:
man_lambert_1 = Maneuver.lambert(ss_earth_1, ss_ven)
ss_trans_1, ss_target_1 = ss_earth_1.apply_maneuver(man_lambert_1, intermediate=True)

man_lambert_2 = Maneuver.lambert(ss_ven, ss_earth_2)
ss_trans_2, ss_target_2 = ss_ven.apply_maneuver(man_lambert_2, intermediate=True)

man_lambert_3 = Maneuver.lambert(ss_earth_2, ss_jup)
ss_trans_3, ss_target_3 = ss_earth_2.apply_maneuver(man_lambert_3, intermediate=True)

In [6]:
#4
plotter = OrbitPlotter3D()
plotter.set_attractor(Sun)

plotter.plot_ephem(earth_1, date_launch, label="Earth_1")
plotter.plot_ephem(ven, date_flyby_Venus, label="Venus")
plotter.plot_ephem(earth_2, date_flyby_Earth, label="Earth_2")
plotter.plot_ephem(jup, date_arrival, label="Jupiter")


plotter.plot_trajectory(
    ss_trans_1.sample(max_anomaly=335 * u.deg), color="blue", label="Transfer orbit E-V"
)

plotter.plot_trajectory(
    ss_trans_2.sample(max_anomaly=280 * u.deg), color="purple", label="Transfer orbit V-E"
)

plotter.plot_trajectory(
    ss_trans_3.sample(max_anomaly=171 * u.deg), color="crimson", label="Transfer orbit E-J"
)

plotter.set_view(30 * u.deg, 260 * u.deg, distance=2 * u.km)

In [7]:
plotter = OrbitPlotter3D()
plotter.set_attractor(Sun)

plotter.plot_ephem(earth_1, date_launch, label="Earth_1")
plotter.plot_ephem(ven, date_flyby_Venus, label="Venus")


plotter.plot_trajectory(
    ss_trans_1.sample(max_anomaly=335 * u.deg), color="blue", label="Transfer orbit E-V"
)

plotter.set_view(30 * u.deg, 260 * u.deg, distance=2 * u.km)

In [8]:
plotter = OrbitPlotter3D()
plotter.set_attractor(Sun)

plotter.plot_ephem(ven, date_flyby_Venus, label="Venus")
plotter.plot_ephem(earth_2, date_flyby_Earth, label="Earth_2")

plotter.plot_trajectory(
    ss_trans_2.sample(max_anomaly=280 * u.deg), color="purple", label="Transfer orbit V-E"
)

plotter.set_view(30 * u.deg, 260 * u.deg, distance=2 * u.km)

In [9]:
plotter = OrbitPlotter3D()
plotter.set_attractor(Sun)

plotter.plot_ephem(earth_2, date_flyby_Earth, label="Earth_2")
plotter.plot_ephem(jup, date_arrival, label="Jupiter")

plotter.plot_trajectory(
    ss_trans_3.sample(max_anomaly=171 * u.deg), color="crimson", label="Transfer orbit E-J"
)

plotter.set_view(30 * u.deg, 260 * u.deg, distance=2 * u.km)

In [44]:
r_earth, v_earth = Ephem.from_body(Earth, date_launch).rv(date_launch)
r_ven, v_ven = Ephem.from_body(Venus, date_flyby_Venus).rv(date_flyby_Venus)
r_earth_2, v_earth_2 = Ephem.from_body(Earth, date_flyby_Earth).rv(date_flyby_Earth)
r_jup, v_jup = Ephem.from_body(Jupiter, date_arrival).rv(date_arrival)

In [62]:
# Фазовые координаты Земли_1

print (f"{r_earth [0]}")
print (f"{r_earth [1]}")
print (f"{r_earth [2]}"'\n')

print (v_earth.to(u.km / u.s))

-37230992.738869734 km
-135843674.5484786 km
-58867657.76067638 km

[28.41635432 -6.75783283 -2.93099165] km / s


In [63]:
# Фазовые координаты Венеры

print (f"{r_ven [0]}")
print (f"{r_ven [1]}")
print (f"{r_ven [2]}"'\n')

print (v_ven.to(u.km / u.s))

27651490.0599603 km
94941430.94866474 km
40988855.195958 km

[-33.94359434   7.30520415   5.43498104] km / s


In [64]:
# Фазовые координаты Земли_2

print (f"{r_earth_2 [0]}")
print (f"{r_earth_2 [1]}")
print (f"{r_earth_2 [2]}"'\n')

print (v_earth_2.to(u.km / u.s))

146659954.04013053 km
-32672756.315061446 km
-14158749.936872052 km

[ 6.50484274 26.46926919 11.47410067] km / s


In [66]:
# Фазовые координаты Юпитера

print (f"{r_jup [0]}")
print (f"{r_jup [1]}")
print (f"{r_jup [2]}"'\n')

print (v_jup.to(u.km / u.s))

-342948768.8919322 km
-668359458.601202 km
-278118816.0992367 km

[11.65419081 -4.47899315 -2.20351138] km / s


In [41]:
nh_tof_1 = date_flyby_Venus - date_launch # время полета Земля1 - Венера

(v0_Ea1, vk_Ven), = iod.lambert(Sun.k, r_earth, r_ven, nh_tof_1)
nh_tof_1

<TimeDelta object: scale='tdb' format='jd' value=166.99999997818995>

In [43]:
# Гиперболический избыток скорости при отлете от Земли

V_inf = (norm(v0_Ea1 - v_earth)).to(u.km / u.s)
V_inf

<Quantity 4.54578181 km / s>

In [12]:
# Гиперболический избыток скорости при отлете от Земли

V_inf_vec = (v0_Ea1 - v_earth).to(u.km / u.s)
V_inf_vec

<Quantity [-3.29935228, -0.95077046, -2.97900022] km / s>

In [13]:
# Гиперболический избыток скорости при подлете к Венере

V_inf_end = (norm(vk_Ven - v_ven)).to(u.km / u.s)
V_inf_end

<Quantity 4.31337537 km / s>

In [14]:
# Гиперболический избыток скорости при подлете к Венере

V_inf_end_vec = (vk_Ven - v_ven).to(u.km / u.s)
V_inf_end_vec

<Quantity [-3.58131835, -2.37541931,  0.36979591] km / s>

In [15]:
nh_tof_2 = date_flyby_Earth - date_flyby_Venus # время полета Венера - Земля2

(v0_Ven, vk_Ea2), = iod.lambert(Sun.k, r_ven, r_earth_2, nh_tof_2)
nh_tof_2

<TimeDelta object: scale='tdb' format='jd' value=657.9999999957513>

In [16]:
# Гиперболический избыток скорости при отлете от Венеры

V_inf_end_n = (norm(v0_Ven - v_ven)).to(u.km / u.s)
V_inf_end_n

<Quantity 9.16235951 km / s>

In [17]:
# Гиперболический избыток скорости при отлете от Венеры

V_inf_end_n_vec = (v0_Ven - v_ven).to(u.km / u.s)
V_inf_end_n_vec

<Quantity [-7.66475362,  5.01902353, -0.09892729] km / s>

In [18]:
# Гиперболический избыток скорости при подлете к Земли_2

V_inf_end_2 = (norm(vk_Ea2 - v_earth_2)).to(u.km / u.s)
V_inf_end_2

<Quantity 14.79794094 km / s>

In [19]:
# Гиперболический избыток скорости при подлете к Земли_2

V_inf_st_2_vec = (vk_Ea2 - v_earth_2).to(u.km / u.s)
V_inf_st_2_vec

<Quantity [-13.984843  ,   4.45622515,   1.88289138] km / s>

In [20]:
nh_tof_3 = date_arrival - date_flyby_Earth # время полета Земля2 - Юпитер

(V0_Ea2, Vk_Jup), = iod.lambert(Sun.k, r_earth_2, r_jup, nh_tof_3)
nh_tof_3

<TimeDelta object: scale='tdb' format='jd' value=769.9999999984603>

In [21]:
# Гиперболический избыток скорости при отлете от Земли_2

V_inf_3_k_0 = (norm(V0_Ea2 - v_earth_2)).to(u.km / u.s)
V_inf_3_k_0

<Quantity 23.22031424 km / s>

In [22]:
# Гиперболический избыток скорости при отлете от Земли_2

V_inf_3_k_0_vec = (V0_Ea2 - v_earth_2).to(u.km / u.s)
V_inf_3_k_0_vec

<Quantity [-22.40365099,   5.75120728,   2.04524577] km / s>

In [23]:
# Гиперболический избыток скорости при подлете к Юпитеру

V_inf_3_k = (norm(Vk_Jup - v_jup)).to(u.km / u.s)
V_inf_3_k

<Quantity 8.15931178 km / s>

In [24]:
V_inf_3_k_vec = (Vk_Jup - v_jup).to(u.km / u.s)
V_inf_3_k_vec

<Quantity [-8.11146995, -0.88094653, -0.04855136] km / s>

In [25]:
time_adv = nh_tof_1 + nh_tof_2 + nh_tof_3 
time_adv

<TimeDelta object: scale='tdb' format='jd' value=1594.9999999724016>

In [26]:
scalar_EV = r_earth[0]*r_ven[0] + r_earth[1]*r_ven[1] +r_earth[2]*r_ven[2] 
scalar_VE_2 = r_earth_2[0]*r_ven[0] + r_earth_2[1]*r_ven[1] +r_earth_2[2]*r_ven[2]
scalar_E2J = r_earth_2[0]*r_jup[0] + r_earth_2[1]*r_jup[1] +r_earth_2[2]*r_jup[2]

In [27]:
fi_EV = (scalar_EV/norm(r_earth)/norm(r_ven))
fi_VE2 = (scalar_VE_2/norm(r_earth_2)/norm(r_ven))
fi_E2J = (scalar_E2J/norm(r_earth_2)/norm(r_jup))

In [28]:
np.degrees(np.arccos(fi_EV))

<Quantity 179.14218246 deg>

In [29]:
np.degrees(np.arccos(fi_VE2))

<Quantity 88.67695268 deg>

In [30]:
np.degrees(np.arccos(fi_E2J))

<Quantity 101.70296446 deg>

In [31]:
## Необходимый импульс на опорной орбите 
from math import*

mu = 398600.4415 ## Гравитационный параметр Земли 
r = 6371.
R = r + 200.
V_gip = 4.5457818 ## Гиперболический избыток скорости при отлете от Земли

dV = sqrt (2 * mu / R + V_gip**2 ) - sqrt(mu/r)
dV * u.km / u.s

<Quantity 4.00596281 km / s>

In [32]:
## Необходимый импульс для торможения на эллиптическую орбиту Юпитера 
## Не уверен что это нужно

muJ = 126686534
r_p = 670900
r_a = 4000000
V_inf_jup = 8.1593118

print (sqrt ((2 * mu / R + V_inf_jup**2 )))
print (sqrt ((2*muJ*r_a)/(r_p*(r_a+r_p))))

dV_el = sqrt ((2 * mu / R + V_inf_jup**2 )) - sqrt ((2*muJ*r_a)/(r_p*(r_a+r_p)))
dV_el * u.km / u.s

13.707496457867549
17.983782441612995


<Quantity -4.27628598 km / s>

In [33]:
## параметры участка Землия-Венера 

from math import*

to_rad = pi/180

ist_an_1_EV = 151.5433336 * to_rad
ist_an_2_EV = 331.6020454 * to_rad

exs_EV = (norm(r_ven)/norm(r_earth) - 1) / (cos(ist_an_1_EV) - (norm(r_ven)/norm(r_earth) * cos(ist_an_2_EV)))
p_EV = norm(r_earth)*(1+cos(ist_an_1_EV)*exs_EV)
a_EV = p_EV/(1 - exs_EV*exs_EV)

print (f"Эксцентриситет {exs_EV}")
print (f"Фокальный параметр {p_EV}")
print (f"Большая полуось {a_EV}")

Эксцентриситет 0.19973500770908048
Фокальный параметр 125852447.07367316 km
Большая полуось 131081835.42501312 km


In [34]:
## параметры участка Венера-Землия_2

ist_an_1_VE2 = 10.1178711 * to_rad
ist_an_2_VE2 = 280.2834074 * to_rad

exs_VE2 = (norm(r_earth_2)/norm(r_ven) - 1) / (cos(ist_an_1_VE2) - (norm(r_earth_2)/norm(r_ven) * cos(ist_an_2_VE2)))
p_VE2 = norm(r_ven)*(1+cos(ist_an_1_VE2)*exs_VE2)
a_VE2 = p_VE2/(1 - exs_VE2*exs_VE2)

print (f"Эксцентриситет {exs_VE2}")
print (f"Фокальный параметр {p_VE2}")
print (f"Большая полуось {a_VE2}")

Эксцентриситет 0.5593745662490762
Фокальный параметр 165991574.98230714 km
Большая полуось 241582814.90385926 km


In [35]:
## параметры участка Землия_2 - Юпитер

ist_an_1_E2J = 272.9243781 * to_rad
ist_an_2_E2J = 170.9408884 * to_rad

exs_E2J = (norm(r_jup)/norm(r_earth_2) - 1) / (cos(ist_an_1_E2J) - (norm(r_jup)/norm(r_earth_2) * cos(ist_an_2_E2J)))
p_E2J = norm(r_earth_2)*(1+cos(ist_an_1_E2J)*exs_E2J)
a_E2J = p_E2J/(1 - exs_E2J*exs_E2J)

print (f"Эксцентриситет {exs_E2J}")
print (f"Фокальный параметр {p_E2J}")
print (f"Большая полуось {a_E2J}")

Эксцентриситет 0.8139233182460377
Фокальный параметр 157187846.256808 km
Большая полуось 465701982.60768193 km


In [36]:
## Вектор гиперболического избытка скорости после грав. маневра у Венеры

r_pi_ven = 8180.5623
muVen = 324859

V_inf_Ven = 4.3133754
V_inf_end_vec_t_0 = -3.5813184
V_inf_end_vec_t_1 = -2.3754193
V_inf_end_vec_t_2 = 0.36979591


betta_ven = 2*asin(1/(1+((r_pi_ven*V_inf_Ven*V_inf_Ven)/muVen)))
gamma_ven = pi
Vxy_inf_ven = sqrt (V_inf_end_vec_t_0**2 + V_inf_end_vec_t_1**2)

V_grav_X_Ven = V_inf_end_vec_t_0 * cos(betta_ven) - (V_inf_end_vec_t_1*V_inf_Ven)/Vxy_inf_ven * sin(betta_ven)*cos(gamma_ven) - (V_inf_end_vec_t_0*V_inf_end_vec_t_2)/Vxy_inf_ven * sin(betta_ven)*sin(gamma_ven)
V_grav_Y_Ven = V_inf_end_vec_t_1 * cos(betta_ven) + (V_inf_end_vec_t_0*V_inf_Ven)/Vxy_inf_ven * sin(betta_ven)*cos(gamma_ven) - (V_inf_end_vec_t_1*V_inf_end_vec_t_2)/Vxy_inf_ven * sin(betta_ven)*sin(gamma_ven)
V_grav_Z_Ven = V_inf_end_vec_t_2 * cos(betta_ven) + Vxy_inf_ven * sin(betta_ven)*sin(gamma_ven)
V_grav_Ven = sqrt (V_grav_X_Ven**2 + V_grav_Y_Ven**2 + V_grav_Z_Ven**2) 

print (f"betta_ven      {betta_ven*180/pi} deg")
print (f"V_grav_X_Ven   {V_grav_X_Ven * u.km / u.s}")
print (f"V_grav_Y_Ven   {V_grav_Y_Ven * u.km / u.s}")
print (f"V_grav_Z_Ven   {V_grav_Z_Ven * u.km / u.s}")
print (f"V_grav_Ven     {V_grav_Ven * u.km / u.s}")

betta_ven      85.83747295030639 deg
V_grav_X_Ven   -2.6378615701886283 km / s
V_grav_Y_Ven   3.4126489984111297 km / s
V_grav_Z_Ven   0.026841973214704343 km / s
V_grav_Ven     4.313375400004103 km / s


In [37]:
## Вектор гиперболического избытка скорости после грав. маневра у Земли_2

r_pi_Ear = 7504.7566
muEar = 398600

V_inf_Ear = 14.797941
V_inf_Ear_vec_t_0 = -13.984843
V_inf_Ear_vec_t_1 = 4.45622523
V_inf_Ear_vec_t_2 = 1.8828914

betta_Ear = 2*asin(1/(1+((r_pi_Ear*V_inf_Ear*V_inf_Ear)/muEar)))
gamma_Ear = pi
Vxy_inf_Ear = sqrt (V_inf_Ear_vec_t_0**2 + V_inf_Ear_vec_t_1**2)

V_grav_X_Ear = V_inf_Ear_vec_t_0 * cos(betta_Ear) - (V_inf_Ear_vec_t_1*V_inf_Ear)/Vxy_inf_Ear * sin(betta_Ear)*cos(gamma_Ear) - (V_inf_Ear_vec_t_0*V_inf_Ear_vec_t_2)/Vxy_inf_Ear * sin(betta_Ear)*sin(gamma_Ear)
V_grav_Y_Ear = V_inf_Ear_vec_t_1 * cos(betta_Ear) + (V_inf_Ear_vec_t_0*V_inf_Ear)/Vxy_inf_Ear * sin(betta_Ear)*cos(gamma_Ear) - (V_inf_Ear_vec_t_1*V_inf_Ear_vec_t_2)/Vxy_inf_Ear * sin(betta_Ear)*sin(gamma_Ear)
V_grav_Z_Ear = V_inf_Ear_vec_t_2 * cos(betta_Ear) + Vxy_inf_Ear * sin(betta_Ear)*sin(gamma_Ear)
V_grav_Ear = sqrt (V_grav_X_Ear**2 + V_grav_Y_Ear**2 + V_grav_Z_Ear**2) 

print (f"betta_Ear      {betta_Ear*180/pi} deg")
print (f"V_grav_X_Ear   {V_grav_X_Ear * u.km / u.s}")
print (f"V_grav_Y_Ear   {V_grav_Y_Ear * u.km / u.s}")
print (f"V_grav_Z_Ear   {V_grav_Z_Ear * u.km / u.s}")
print (f"V_grav_Ear     {V_grav_Ear * u.km / u.s}")

betta_Ear      22.513078877943265 deg
V_grav_X_Ear   -11.19884266951259 km / s
V_grav_Y_Ear   9.515222889107811 km / s
V_grav_Z_Ear   1.7394003011988357 km / s
V_grav_Ear     14.797940977504528 km / s
